# Code-Hotspot-Analyse (leer)
Step-by-Step-Tutorial


## Rohdaten via Kommandozeile

### Lines of Code ("Complexity")

Variante mit `cloc`

### Änderungshäufigkeiten ("Hotness")

Änderungen pro Datei mit `git log`

## Datenmassage mit Data-Science-Werkzeugen

### Lines of Code

"Lines of Code"-Daten einlesen

In der Theorie: Datei mit Zeilenanzahlen direkt einlesen

In der Praxis: Einlesen mit kleineren Anpassungen

### Änderungsdaten

Blick auf die Rohdaten

Datei mit jeder geänderten Datei einlesen (+ Spaltennamen)

Hier noch zu erledigen: Änderungshäufigkeit der Dateien zählen

Getrennte Daten zusammenführen

## Visualisierung der Daten

Woher nehmen?
[https://observablehq.com/@d3/gallery](https://observablehq.com/@d3/gallery)

Wir brauchen folgendes Datenformat und ein D3-Visualisierungs-Template für Circle Pack Hierarchy Chart.

- Komplette Datei: <code><a href="demo/spring_petclinic_demo_flare.json" target="_blank">demo/spring_petclinic_demo_flare.json</a></code>
- Template: <code><a href="vis/template_circle_pack_hierarchy_chart_d3_inline.html" target="_blank">vis/template_circle_pack_hierarchy_chart_d3_inline.html</a></code> (hier: Quellcode anzeigen lassen)

<small><pre>{
    'name': 'flare',
    'children': [{
            'name': 'src',
            'children': [{
                    'name': 'main',
                    'children': [{
                            'name': 'java',
                            'children': [{
                                    'name': 'org',
                                    'children': [{
                                            'name': 'springframework',
                                            'children': [{
                                                    'name': 'samples',
                                                    'children': [{
                                                            'name': 'petclinic',
                                                            'children': [{
                                                                    'name': 'repository',
                                                                    'children': [{
                                                                            'name': 'jdbc',
                                                                            'children': [{
                                                                                    'name': 'JdbcOwnerRepositoryImpl.java (158.0 [27])',
                                                                                    'size': 158.0,
                                                                                    'color': '#b40426'
                                                                                }, {
                                                                                    'name': 'JdbcVetRepositoryImpl.java (88.0 [20])',
                                                                                    'size': 88.0,
                                                                                    'color': '#f59d7e'
                                                                                }, {
                                                                                    'name': 'JdbcVisitRepositoryImpl.java (103.0 [19])',
                                                                                    'size': 103.0,
                                                                                    'color': '#f7aa8c'
                                                                                },
...
</pre></small>

Daten für Visualisierung aufbereiten: In D3-Datenformat (JSON) umwandeln, HTML-Template für Circle Pack Hierarchy Chart mit Daten füttern und Datei erzeugen (nein, diesen Code schreiben wir nicht selbst heute...).

In [ ]:
from matplotlib import cm
from matplotlib.colors import rgb2hex
import json
from IPython.core.display import HTML


def create_plot_data(df, color_column_name, size_column_name, seperator):
    plot_data = pd.DataFrame(index=df.index)
    plot_data['value_for_color'] = df[color_column_name]
    plot_data['ratio_for_color'] = plot_data['value_for_color'] / plot_data['value_for_color'].max()
    plot_data['color'] = plot_data['ratio_for_color'].apply(lambda x : rgb2hex(cm.coolwarm(x)))
    plot_data['size'] = df[size_column_name]
    plot_data[['path', 'name']] = df.index.str.rsplit(seperator, n=1).to_list()
    plot_data['path_list'] = plot_data['path'].str.split(seperator)
    return plot_data

def create_flare_json(df):

    json_data = {'name': 'flare', 'children': []}

    for _, series in df.iterrows():
        hierarchical_data = series['path_list']

        children = json_data['children']
        for part in hierarchical_data:
            entry = next((child for child in children if child.get('name', '') == part), None)
            if not entry:
                entry = {'name': part, 'children': []}
                children.append(entry)
            children = entry['children']

        children.append({
            'name': f"{series['name']} ({series['size']} [{series['value_for_color']}])",
            'size': series['size'],
            'color': series['color']
        })

    return json_data


def create_file(hotspots, color_column_name, size_column_name, separator, suffix=""):
    json_data = create_flare_json(create_plot_data(hotspots, color_column_name, size_column_name,separator))
            
    with open("vis/template_circle_pack_hierarchy_chart_d3_inline.html") as html_template:
        html = html_template.read().replace("###JSON###", str(json_data))
        
        with open(f'output/code_hotspots{suffix}.html', mode='w') as html_out:
            html_out.write(html)
    
    return HTML(f'<a href="output/code_hotspots{suffix}.html" target="_blank">Hotspots via Circle Pack Hierarchy Chart {suffix}</a>')
            
create_file(hotspots, "changes", "code", "/")

## Herausforderung
In `data/spring_petclinic_production_coverage_data.csv` liegt eine Datei mit Messergebnissen in Form einer Code Coverage, welche während einer repräsentativen Nutzung der Anwendung aufgezeichnet wurde. Visualisiert diese Daten mit Hilfe der Circle Pack Hierarchy Chart.

Daten einlesen mittels pandas

Berechne `ratio`, also den Prozentsatz des durchgelaufenen Codes.

Erzeuge eine neue Spalte mit dem kompletten Pfad zu einer Quellcodedatei und setze diesen mit `set_index` als Index.

Nutze die obige `create_file`-Funktion, um eine Visualisierung zu erzeugen (Tipp: `suffix` als letzen Parameter der Funktion verwenden, um eine Datei mit anderem Namen zu erzeugen)